In [1]:
from aqlm import QuantizedLinear
from aqlm.utils import _dequantize_weight, unpack_int_data

In [2]:
!bash build.sh

[  2%] Built target pthreadpool
[  8%] Built target cpuinfo_internals
[  8%] Built target cpuinfo
[  9%] Built target gflags_nothreads_static
[ 17%] Built target eigen_blas
[ 29%] Built target flatc
[ 30%] Generating common_schema headers
[ 30%] Built target common_schema
[ 30%] Generating program_schema headers
[ 30%] Built target program_schema
[ 30%] Building CXX object /Users/blacksamorez/reps/AQLM/inference_lib/src/aqlm/inference_kernels/executorch/CMakeFiles/executorch_no_prim_ops.dir/runtime/executor/method_meta.cpp.o
[ 30%] Building CXX object /Users/blacksamorez/reps/AQLM/inference_lib/src/aqlm/inference_kernels/executorch/CMakeFiles/executorch_no_prim_ops.dir/runtime/executor/method.cpp.o
[ 31%] Building CXX object /Users/blacksamorez/reps/AQLM/inference_lib/src/aqlm/inference_kernels/executorch/CMakeFiles/executorch_no_prim_ops.dir/runtime/executor/program.cpp.o
[ 31%] Building CXX object /Users/blacksamorez/reps/AQLM/inference_lib/src/aqlm/inference_kernels/executorch/CMake

In [3]:
SIZE = 1024

layer = QuantizedLinear(
    in_features=SIZE,
    out_features=SIZE * 3,
    in_group_size=8,
    out_group_size=1,
    num_codebooks=2,
    nbits_per_codebook=8,
    bias=True,
)

In [4]:
reference_weight = _dequantize_weight(
    unpack_int_data(layer.codes, 8),
    layer.codebooks,
    layer.scales,
)

In [5]:
import torch

input = torch.rand((1, 1, SIZE)) * 2 - 1

In [6]:
import torch
torch.ops.load_library("./cmake-out/libaqlm.dylib")

reference = input @ reference_weight.T + layer.bias
test = torch.ops.aqlm.code2x8_lut_matmat(
    input,
    torch.permute(layer.codes, (1, 0, 2)).contiguous(),
    layer.codebooks,
    layer.scales,
    layer.bias,
)

torch.testing.assert_close(
    test,
    reference,
    atol=0.01,
    rtol=1e-3,
)

In [7]:
%%time

for i in range(10):
    input @ reference_weight.T

CPU times: user 2.86 ms, sys: 3.22 ms, total: 6.08 ms
Wall time: 1.19 ms


In [8]:
%%time

for i in range(10):
    torch.ops.aqlm.code2x8_lut_matmat(
        input,
        torch.permute(layer.codes, (1, 0, 2)).contiguous(),
        layer.codebooks,
        layer.scales,
        layer.bias,
    )

CPU times: user 171 ms, sys: 31.5 ms, total: 202 ms
Wall time: 35.9 ms
